## dummy swapping pmid and log

In [5]:
%reload_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import os
import logging
import time
import uuid
import openai
from pathlib import Path
from llm_explainer.llm_explainer import vllm_get_response
from hgcr_util.text_util import get_abstr_text
from hgcr_util.lazy_json_kv_loader import LazyJsonlAbstractLoader
from llm_context_processor_2 import LLMContextProcessor

In [3]:
df = pd.read_pickle('./all_pkl_files/three_samples.pkl')
df

,pair,subj_name,obj_name,pmid,pmid_text,source,first_year,label,context_pmids,path,context_pmids_space_edge,sorted_context_pmids,final_context_pmids
774,C0076080 C1530270,temozolomide,"IRF1 protein, human",35274103,XAF1 directs glioma response to temozolomide t...,sr_pred_sent_7,2022,1,"{('C0076080', 'C0069197'): ['19514084', '32901...","[C0076080, C0069197, C1530270]","{('C0076080', 'C0069197'): ['4300795', '210665...","{('C0076080', 'C0069197'): ['19514084', '32901...","{('C0076080', 'C0069197'): ['19514084', '32901..."
775,C0076080 C1530270,temozolomide,"IRF1 protein, human",35274103,XAF1 directs glioma response to temozolomide t...,sr_pred_sent_7,2022,1,"{('C0076080', 'C1337027'): ['31575897', '28122...","[C0076080, C1337027, C0069197, C1530270]","{('C0076080', 'C1337027'): ['24214988', '75263...","{('C0076080', 'C1337027'): ['31575897', '28122...","{('C0076080', 'C1337027'): ['31575897', '28122..."
776,C0076080 C1530270,temozolomide,"IRF1 protein, human",35274103,XAF1 directs glioma response to temozolomide t...,sr_pred_sent_7,2022,1,"{('C0076080', 'C1337027'): ['31575897', '28122...","[C0076080, C1337027, C1530270]","{('C0076080', 'C1337027'): ['24214988', '75263...","{('C0076080', 'C1337027'): ['31575897', '28122...","{('C0076080', 'C1337027'): ['31575897', '28122..."
777,C0076080 C1530270,temozolomide,"IRF1 protein, human",35274103,XAF1 directs glioma response to temozolomide t...,sr_pred_sent_7,2022,0,"{('C0076080', 'C2931926'): ['34700192', '32629...","[C0076080, C2931926, C1530270]","{('C0076080', 'C2931926'): ['29436151', '29049...","{('C0076080', 'C2931926'): ['32629176', '26981...","{('C0076080', 'C2931926'): ['32629176', '26981..."
778,C0076080 C1530270,temozolomide,"IRF1 protein, human",35274103,XAF1 directs glioma response to temozolomide t...,sr_pred_sent_7,2022,0,"{('C0076080', 'C1506592'): ['16908595', '33085...","[C0076080, C1506592, C2713871, C1530270]","{('C0076080', 'C1506592'): ['7364749', '268136...","{('C0076080', 'C1506592'): ['23243055', '33085...","{('C0076080', 'C1506592'): ['23243055', '33085..."
779,C0076080 C1530270,temozolomide,"IRF1 protein, human",35274103,XAF1 directs glioma response to temozolomide t...,sr_pred_sent_7,2022,0,"{('C0076080', 'C0123707'): ['25806680', '30562...","[C0076080, C0123707, C0033308, C1530270]","{('C0076080', 'C0123707'): ['33469032', '27696...","{('C0076080', 'C0123707'): ['29456350', '30562...","{('C0076080', 'C0123707'): ['29456350', '30562..."
1386,C0011812 C0663241,dextroamphetamine,linezolid,37295073,A Case Series on Serotonin Syndrome from Conco...,title_cooc,2023,1,"{('C0011812', 'C0010620'): ['414256', '139615'...","[C0011812, C0010620, C0024586, C0663241]","{('C0011812', 'C0010620'): ['805443', '1408460...","{('C0011812', 'C0010620'): ['7195140', '661046...","{('C0011812', 'C0010620'): ['7195140', '661046..."
1387,C0011812 C0663241,dextroamphetamine,linezolid,37295073,A Case Series on Serotonin Syndrome from Conco...,title_cooc,2023,1,"{('C0011812', 'C0024586'): ['6463096', '173646...","[C0011812, C0024586, C0663241]","{('C0011812', 'C0024586'): ['5786556', '764914...","{('C0011812', 'C0024586'): ['17364639', '96927...","{('C0011812', 'C0024586'): ['17364639', '96927..."
1388,C0011812 C0663241,dextroamphetamine,linezolid,37295073,A Case Series on Serotonin Syndrome from Conco...,title_cooc,2023,1,"{('C0011812', 'C0010620'): ['139615'], ('C0010...","[C0011812, C0010620, C0663241]","{('C0011812', 'C0010620'): ['805443', '1408460...","{('C0011812', 'C0010620'): ['139615'], ('C0010...","{('C0011812', 'C0010620'): ['139615', '805443'..."
1389,C0011812 C0663241,dextroamphetamine,linezolid,37295073,A Case Series on Serotonin Syndrome from Conco...,title_cooc,2023,0,"{('C0011812', 'C0004609'): ['24346808', '73674...","[C0011812, C0004609, C0663241]","{('C0011812', 'C0004609'): ['66577', '22704747...","{('C0011812', 'C0004609'): ['19042205', '11271...","{('C0011812', 'C0004609'): ['19042205', '11271..."


In [5]:
len(df)

18

In [4]:
sents_db = LazyJsonlAbstractLoader
sent_jsonl_dir = Path(
    "/work/acslab/shared/Agatha_shared/"
    "pmid_to_sent_id_w_text_kv_jsonl_lazy_chunks"
)

sents_db = LazyJsonlAbstractLoader(
    list(sent_jsonl_dir.glob("*.jsonl"))
)

Indexing jsonl files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 109/109 [00:12<00:00,  8.48it/s]


## NOTE: for 3 pair with 6 different path labeled 0 and 1 for llama3.3_70B it took 40 minutes max_iteration=4 number of pmid per edge=5:

In [6]:
try:
    processor = LLMContextProcessor(sents_db=sents_db)
    updated_df = processor.process_df_with_llm(
        df=df,
        max_iterations=4,
        k=5
    )
    updated_df.to_pickle("./all_pkl_files/mini_sample_3_pair_k_5_max_itr4_llama33_70B.pkl")

except RuntimeError as e:
    print(f"Critical error occurred: {e}")
    processor.logger.error(f"Critical error occurred: {e}")

In [7]:
res_df_mini_sample_3_pair_k_5_max_itr4_llama33_70B = pd.read_pickle('./all_pkl_files/mini_sample_3_pair_k_5_max_itr4_llama33_70B.pkl')
res_df_mini_sample_3_pair_k_5_max_itr4_llama33_70B

,pair,subj_name,obj_name,pmid,pmid_text,source,first_year,label,context_pmids,path,context_pmids_space_edge,sorted_context_pmids,final_context_pmids,respond_llm,respond_time_llm,prompts_per_iteration,pmid_contexts_per_iteration,pmid_changes_per_iteration
774,C0076080 C1530270,temozolomide,"IRF1 protein, human",35274103,XAF1 directs glioma response to temozolomide t...,sr_pred_sent_7,2022,1,"{('C0076080', 'C0069197'): ['19514084', '32901...","[C0076080, C0069197, C1530270]","{('C0076080', 'C0069197'): ['4300795', '210665...","{('C0076080', 'C0069197'): ['19514084', '32901...","{('C0076080', 'C0069197'): ['19514084', '32901...","[{'iteration': 1, 'response': 'Based on the pr...","[33.195026874542236, 31.69631314277649, 35.408...","[Based on the following scientific abstracts, ...","[{('C0076080', 'C0069197'): ['19514084', '3290...","[{'edge': ('C0069197', 'C1530270'), 'dropped':..."
775,C0076080 C1530270,temozolomide,"IRF1 protein, human",35274103,XAF1 directs glioma response to temozolomide t...,sr_pred_sent_7,2022,1,"{('C0076080', 'C1337027'): ['31575897', '28122...","[C0076080, C1337027, C0069197, C1530270]","{('C0076080', 'C1337027'): ['24214988', '75263...","{('C0076080', 'C1337027'): ['31575897', '28122...","{('C0076080', 'C1337027'): ['31575897', '28122...","[{'iteration': 1, 'response': 'To describe an ...","[34.983481645584106, 36.60644602775574, 30.894...","[Based on the following scientific abstracts, ...","[{('C0076080', 'C1337027'): ['31575897', '2812...","[{'edge': ('C1337027', 'C0069197'), 'dropped':..."
776,C0076080 C1530270,temozolomide,"IRF1 protein, human",35274103,XAF1 directs glioma response to temozolomide t...,sr_pred_sent_7,2022,1,"{('C0076080', 'C1337027'): ['31575897', '28122...","[C0076080, C1337027, C1530270]","{('C0076080', 'C1337027'): ['24214988', '75263...","{('C0076080', 'C1337027'): ['31575897', '28122...","{('C0076080', 'C1337027'): ['31575897', '28122...","[{'iteration': 1, 'response': 'Based on the pr...","[26.065773248672485, 28.8979172706604, 33.3238...","[Based on the following scientific abstracts, ...","[{('C0076080', 'C1337027'): ['31575897', '2812...","[{'edge': ('C0076080', 'C1337027'), 'dropped':..."
777,C0076080 C1530270,temozolomide,"IRF1 protein, human",35274103,XAF1 directs glioma response to temozolomide t...,sr_pred_sent_7,2022,0,"{('C0076080', 'C2931926'): ['34700192', '32629...","[C0076080, C2931926, C1530270]","{('C0076080', 'C2931926'): ['29436151', '29049...","{('C0076080', 'C2931926'): ['32629176', '26981...","{('C0076080', 'C2931926'): ['32629176', '26981...","[{'iteration': 1, 'response': 'To describe an ...","[41.17671751976013, 28.509257316589355, 34.484...","[Based on the following scientific abstracts, ...","[{('C0076080', 'C2931926'): ['32629176', '2698...","[{'edge': ('C0076080', 'C2931926'), 'dropped':..."
778,C0076080 C1530270,temozolomide,"IRF1 protein, human",35274103,XAF1 directs glioma response to temozolomide t...,sr_pred_sent_7,2022,0,"{('C0076080', 'C1506592'): ['16908595', '33085...","[C0076080, C1506592, C2713871, C1530270]","{('C0076080', 'C1506592'): ['7364749', '268136...","{('C0076080', 'C1506592'): ['23243055', '33085...","{('C0076080', 'C1506592'): ['23243055', '33085...","[{'iteration': 1, 'response': 'Based on the pr...","[39.19556903839111, 42.19934821128845, 31.7485...","[Based on the following scientific abstracts, ...","[{('C0076080', 'C1506592'): ['23243055', '3308...","[{'edge': ('C0076080', 'C1506592'), 'dropped':..."
779,C0076080 C1530270,temozolomide,"IRF1 protein, human",35274103,XAF1 directs glioma response to temozolomide t...,sr_pred_sent_7,2022,0,"{('C0076080', 'C0123707'): ['25806680', '30562...","[C0076080, C0123707, C0033308, C1530270]","{('C0076080', 'C0123707'): ['33469032', '27696...","{('C0076080', 'C0123707'): ['29456350', '30562...","{('C0076080', 'C0123707'): ['29456350', '30562...","[{'iteration': 1, 'response': 'Based on the pr...","[34.53022503852844, 37.267558336257935, 34.992...","[Based on the following scientific a

In [8]:
res_df_mini_sample_3_pair_k_5_max_itr4_llama33_70B[['respond_llm',	'respond_time_llm',	'prompts_per_iteration',	'pmid_contexts_per_iteration',	'pmid_changes_per_iteration']]

,respond_llm,respond_time_llm,prompts_per_iteration,pmid_contexts_per_iteration,pmid_changes_per_iteration
774,"[{'iteration': 1, 'response': 'Based on the pr...","[33.195026874542236, 31.69631314277649, 35.408...","[Based on the following scientific abstracts, ...","[{('C0076080', 'C0069197'): ['19514084', '3290...","[{'edge': ('C0069197', 'C1530270'), 'dropped':..."
775,"[{'iteration': 1, 'response': 'To describe an ...","[34.983481645584106, 36.60644602775574, 30.894...","[Based on the following scientific abstracts, ...","[{('C0076080', 'C1337027'): ['31575897', '2812...","[{'edge': ('C1337027', 'C0069197'), 'dropped':..."
776,"[{'iteration': 1, 'response': 'Based on the pr...","[26.065773248672485, 28.8979172706604, 33.3238...","[Based on the following scientific abstracts, ...","[{('C0076080', 'C1337027'): ['31575897', '2812...","[{'edge': ('C0076080', 'C1337027'), 'dropped':..."
777,"[{'iteration': 1, 'response': 'To describe an ...","[41.17671751976013, 28.509257316589355, 34.484...","[Based on the following scientific abstracts, ...","[{('C0076080', 'C2931926'): ['32629176', '2698...","[{'edge': ('C0076080', 'C2931926'), 'dropped':..."
778,"[{'iteration': 1, 'response': 'Based on the pr...","[39.19556903839111, 42.19934821128845, 31.7485...","[Based on the following scientific abstracts, ...","[{('C0076080', 'C1506592'): ['23243055', '3308...","[{'edge': ('C0076080', 'C1506592'), 'dropped':..."
779,"[{'iteration': 1, 'response': 'Based on the pr...","[34.53022503852844, 37.267558336257935, 34.992...","[Based on the following scientific abstracts, ...","[{('C0076080', 'C0123707'): ['29456350', '3056...","[{'edge': ('C0123707', 'C0033308'), 'dropped':..."
1386,"[{'iteration': 1, 'response': 'Based on the pr...","[29.576554536819458, 35.0388925075531, 33.2695...","[Based on the following scientific abstracts, ...","[{('C0011812', 'C0010620'): ['7195140', '66104...","[{'edge': ('C0011812', 'C0010620'), 'dropped':..."
1387,"[{'iteration': 1, 'response': 'An indirect rel...","[29.309974670410156, 32.61806797981262, 32.936...","[Based on the following scientific abstracts, ...","[{('C0011812', 'C0024586'): ['17364639', '9692...","[{'edge': ('C0024586', 'C0663241'), 'dropped':..."
1388,"[{'iteration': 1, 'response': 'To describe an ...","[29.69477605819702, 30.643377780914307, 20.391...","[Based on the following scientific abstracts, ...","[{('C0011812', 'C0010620'): ['139615', '805443...","[{'edge': ('C0010620', 'C0663241'), 'dropped':..."
1389,"[{'iteration': 1, 'response': 'To describe an ...","[29.05800199508667, 31.147324800491333, 34.714...","[Based on the following scientific abstracts, ...","[{('C0011812', 'C0004609'): ['19042205', '1127...","[{'edge': ('C0011812', 'C0004609'), 'dropped':..."


In [11]:
len(res_df_mini_sample_3_pair_k_5_max_itr4_llama33_70B)

18

In [9]:
def extract_exploded_results_v3(processed_df, output_file=None):
    """
    Extracts and explodes iterative results from the DataFrame generated by LLMContextProcessor.
    Each iteration-related column is exploded to create a new row for each iteration,
    while retaining the original index for easier tracking. Additionally, only the "response" field
    is retained from the "respond_llm" column.

    Args:
        processed_df (pd.DataFrame): DataFrame generated by LLMContextProcessor.
        output_file (str, optional): If provided, saves the exploded DataFrame to this file.

    Returns:
        pd.DataFrame: Exploded DataFrame with a new row for each iteration.
    """
    # Step 1: Ensure all iteration-related columns are aligned
    iteration_columns = [
        "respond_llm", "respond_time_llm", 
        "prompts_per_iteration", "pmid_contexts_per_iteration", "pmid_changes_per_iteration"
    ]
    for col in iteration_columns:
        assert processed_df[col].apply(len).nunique() == 1, f"Column {col} has misaligned iteration lengths."

    # Step 2: Explode iteration-related columns
    exploded_df = processed_df.copy()
    exploded_df = exploded_df.explode("respond_llm").reset_index(names="original_index")
    exploded_df["respond_time_llm"] = exploded_df["respond_time_llm"].explode(ignore_index=True)
    exploded_df["prompts_per_iteration"] = exploded_df["prompts_per_iteration"].explode(ignore_index=True)
    exploded_df["pmid_contexts_per_iteration"] = exploded_df["pmid_contexts_per_iteration"].explode(ignore_index=True)
    exploded_df["pmid_changes_per_iteration"] = exploded_df["pmid_changes_per_iteration"].explode(ignore_index=True)

    # Retain only the "response" field from "respond_llm"
    exploded_df["respond_llm"] = exploded_df["respond_llm"].apply(lambda x: x["response"] if isinstance(x, dict) and "response" in x else None)

    # Step 3: Add iteration number
    exploded_df["iteration"] = exploded_df.groupby("original_index").cumcount() + 1

    # Step 4: Extract detailed information for each iteration's PMIDs
    exploded_df["edge_modified"] = exploded_df["pmid_changes_per_iteration"].apply(lambda x: x["edge"] if pd.notnull(x) else None)
    exploded_df["pmid_dropped"] = exploded_df["pmid_changes_per_iteration"].apply(lambda x: x["dropped"] if pd.notnull(x) else None)
    exploded_df["pmid_added"] = exploded_df["pmid_changes_per_iteration"].apply(lambda x: x["added"] if pd.notnull(x) else None)

    # Step 5: Save the DataFrame if an output file is specified
    if output_file:
        if output_file.endswith(".pkl"):
            exploded_df.to_pickle(output_file)
        elif output_file.endswith(".xlsx"):
            exploded_df.to_excel(output_file, index=False)
        elif output_file.endswith(".csv"):
            exploded_df.to_csv(output_file, index=False)
        else:
            raise ValueError("Unsupported file format. Use '.pkl', '.xlsx', or '.csv'.")

    return exploded_df


In [10]:
exploded_res_df_mini_sample_3_pair_k_5_max_itr4_llama33_70B = extract_exploded_results_v3(res_df_mini_sample_3_pair_k_5_max_itr4_llama33_70B, output_file="exploded_results_exploded_res_df_proc_779_samp_k_5_max_itr_3.csv")
len(exploded_res_df_mini_sample_3_pair_k_5_max_itr4_llama33_70B)

72

In [12]:
exploded_res_df_mini_sample_3_pair_k_5_max_itr4_llama33_70B.columns

Index(['original_index', 'pair', 'subj_name', 'obj_name', 'pmid', 'pmid_text',
       'source', 'first_year', 'label', 'context_pmids', 'path',
       'context_pmids_space_edge', 'sorted_context_pmids',
       'final_context_pmids', 'respond_llm', 'respond_time_llm',
       'prompts_per_iteration', 'pmid_contexts_per_iteration',
       'pmid_changes_per_iteration', 'iteration', 'edge_modified',
       'pmid_dropped', 'pmid_added'],
      dtype='object')

In [13]:
exploded_res_df_mini_sample_3_pair_k_5_max_itr4_llama33_70B

,original_index,pair,subj_name,obj_name,pmid,pmid_text,source,first_year,label,context_pmids,...,final_context_pmids,respond_llm,respond_time_llm,prompts_per_iteration,pmid_contexts_per_iteration,pmid_changes_per_iteration,iteration,edge_modified,pmid_dropped,pmid_added
0,774,C0076080 C1530270,temozolomide,"IRF1 protein, human",35274103,XAF1 directs glioma response to temozolomide t...,sr_pred_sent_7,2022,1,"{('C0076080', 'C0069197'): ['19514084', '32901...",...,"{('C0076080', 'C0069197'): ['19514084', '32901...","Based on the provided abstracts, an indirect r...",33.195027,"Based on the following scientific abstracts, p...","{('C0076080', 'C0069197'): ['19514084', '32901...","{'edge': ('C0069197', 'C1530270'), 'dropped': ...",1,"(C0069197, C1530270)",17102778,2418942
1,774,C0076080 C1530270,temozolomide,"IRF1 protein, human",35274103,XAF1 directs glioma response to temozolomide t...,sr_pred_sent_7,2022,1,"{('C0076080', 'C0069197'): ['19514084', '32901...",...,"{('C0076080', 'C0069197'): ['19514084', '32901...","Based on the provided abstracts, an indirect r...",31.696313,"Based on the following scientific abstracts, p...","{('C0076080', 'C0069197'): ['19514084', '32901...","{'edge': ('C0076080', 'C0069197'), 'dropped': ...",2,"(C0076080, C0069197)",20706757,17097370
2,774,C0076080 C1530270,temozolomide,"IRF1 protein, human",35274103,XAF1 directs glioma response to temozolomide t...,sr_pred_sent_7,2022,1,"{('C0076080', 'C0069197'): ['19514084', '32901...",...,"{('C0076080', 'C0069197'): ['19514084', '32901...","Based on the provided abstracts, an indirect r...",35.408087,"Based on the following scientific abstracts, p...","{('C0076080', 'C0069197'): ['19514084', '32901...","{'edge': ('C0076080', 'C0069197'), 'dropped': ...",3,"(C0076080, C0069197)",32901889,16222105
3,774,C0076080 C1530270,temozolomide,"IRF1 protein, human",35274103,XAF1 directs glioma response to temozolomide t...,sr_pred_sent_7,2022,1,"{('C0076080', 'C0069197'): ['19514084', '32901...",...,"{('C0076080', 'C0069197'): ['19514084', '32901...","Based on the provided abstracts, an indirect r...",28.589508,"Based on the following scientific abstracts, p...","{('C0076080', 'C0069197'): ['19514084', '43007...","{'edge': 'max_iteration', 'dropped': 'max_iter...",4,max_iteration,max_iteration,max_iteration
4,775,C0076080 C1530270,temozolomide,"IRF1 protein, human",35274103,XAF1 directs glioma response to temozolomide t...,sr_pred_sent_7,2022,1,"{('C0076080', 'C1337027'): ['31575897', '28122...",...,"{('C0076080', 'C1337027'): ['31575897', '28122...",To describe an indirect relationship between t...,33.195027,"Based on the following scientific abstracts, p...","{('C0076080', 'C0069197'): ['19514084', '32901...","{'edge': ('C0069197', 'C1530270'), 'dropped': ...",1,"(C0069197, C1530270)",17102778,2418942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,1588,C0002395 C1705607,F3 wt Allele,Alzheimer's Disease,35950105,Quality of dietary fat and risk of Alzheimer's...,sr_pred_sent_9,2022,0,"{('C0002395', 'C1422519'): ['32706201', '25092...",...,"{('C0002395', 'C1422519'): ['25092125', '31878...","Based on the provided scientific abstracts, an...",39.023563,"Based on the following scientific abstracts, p...","{('C0076080', 'C1506592'): ['23243055', '33085...","{'edge': 'max_iteration', 'dropped': 'max_iter...",4,max_iteration,max_iteration,max_iteration
68,1589,C0002395 C1705607,F3 wt Allele,Alzheimer's Disease,35950105,Quality of dietary fat and risk of Alzheimer's...,sr_pred_sent_9,2022,0,"{('C0002395', 'C0202236'): ['31765631', '19504...",...,"{('C0002395', 'C0202236'): ['19504347', '31765...",To describe an indirect relationship between t...,39.195569,"Based on the following scientific abstracts, p...","{('C0076080', 'C1506592'): ['23243055', '33085...","{'edge': ('C0076080', 'C1506592'), 'dropped': ...",1,"(C0076080, C1506592)",26813683,19251725
69,1589,C0002395 C1705607,F3 wt Allele,Alzheimer's Disease,35950105,Quality of die

In [27]:
filtered_df = exploded_res_df_mini_sample_3_pair_k_5_max_itr4_llama33_70B[exploded_res_df_mini_sample_3_pair_k_5_max_itr4_llama33_70B['original_index'] == 774]
columns_to_view = ['final_context_pmids', 'respond_llm', 'pmid_contexts_per_iteration', 'pmid_changes_per_iteration', 'edge_modified','pmid_dropped','pmid_added']
filtered_df = filtered_df[columns_to_view]
filtered_df

,final_context_pmids,respond_llm,pmid_contexts_per_iteration,pmid_changes_per_iteration,edge_modified,pmid_dropped,pmid_added
0,"{('C0076080', 'C0069197'): ['19514084', '32901889', '20706757', '4300795', '21066565', '17097370', '16222105', '25928166', '9473363', '34100706', '24640399', '15358884', '34352381', '25939521', '33787276', '1783274', '35739713', '34553478', '33805730', '17981249', '3963214', '18850102', '3008899', '33564284', '14369208', '7624262', '6773721'], ('C0069197', 'C1530270'): ['16520149', '34932098', '33992628', '17102778', '7199054', '2418942', '14596898', '2683289', '2888564', '1591363', '6481535', '35604826', '2332516', '2480183', '34621426', '33511939', '34763534', '34759615', '7510792', '16599639', '2411414', '3471909']}","Based on the provided abstracts, an indirect relationship between temozolomide and IRF1 protein, human, can be proposed as follows:\n\n1. **Temozolomide's mechanism of action**: Temozolomide is a DNA alkylating agent that works by damaging the DNA of cancer cells, thereby inhibiting their growth and proliferation. The effectiveness of temozolomide is influenced by the activity of the O6-methylguanine-DNA methyltransferase (MGMT) enzyme, which can repair the DNA damage caused by temozolomide.\n\n2. **MGMT and IRF1 connection**: The abstract ""Methionine-stress: a pleiotropic approach in enhancing the efficacy of chemotherapy"" mentions that methionine stress can inhibit the growth of brain tumor xenografts and enhance the efficacy of DNA alkylating chemotherapeutic agents, including temozolomide. This is associated with the downregulation of MGMT expression. Additionally, methionine stress upregulates IRF-1, a transcription factor involved in cell cycle regulation, apoptosis, and immune responses.\n\n3. **IRF1's role in cell cycle regulation and apoptosis**: IRF1 is known to regulate the expression of genes involved in cell cycle arrest, apoptosis, and DNA repair. Its upregulation can lead to the inhibition of cell cycle progression and the induction of apoptosis in cancer cells.\n\n4. **Temozolomide's effect on IRF1**: Although there is no direct mention of temozolomide's effect on IRF1 in the provided abstracts, it can be inferred that temozolomide's ability to induce DNA damage and activate cell cycle checkpoints may lead to the upregulation of IRF1. This is because IRF1 is involved in the regulation of cell cycle and apoptosis in response to DNA damage.\n\n5. **Indirect relationship between temozolomide and IRF1**: Based on the above points, an indirect relationship between temozolomide and IRF1 can be proposed. Temozolomide's DNA damaging effects can lead to the upregulation of IRF1, either directly or through the activation of cell cycle checkpoints and DNA repair mechanisms. The upregulation of IRF1 can, in turn, contribute to the inhibition of cancer cell growth and proliferation, enhancing the efficacy of temozolomide.\n\nIn summary, the indirect relationship between temozolomide and IRF1 protein, human, involves the following steps:\n\nTemozolomide → DNA damage → Cell cycle checkpoint activation → IRF1 upregulation → Inhibition of cancer cell growth and proliferation\n\nThis proposed relationship highlights the potential for IRF1 to play a role in the mechanism of action of temozolomide, and further research is needed to fully elucidate this connection.","{('C0076080', 'C0069197'): ['19514084', '32901889', '20706757', '4300795', '21066565'], ('C0069197', 'C1530270'): ['16520149', '34932098', '33992628', '17102778', '7199054']}","{'edge': ('C0069197', 'C1530270'), 'dropped': '17102778', 'added': '2418942'}","(C0069197, C1530270)",17102778,2418942
1,"{('C0076080', 'C0069197'): ['19514084', '32901889', '20706757', '4300795', '21066565', '17097370', '16222105', '25928166', '9473363', '34100706', '24640399', '15358884', '34352381', '25939521', '33787276', '1783274', '35739713', '34553478', '33805730', '17981249', '3963214', '18850102', '3008899', '33564284', '14369208', '7624262'

In [17]:
exploded_res_df_mini_sample_3_pair_k_5_max_itr4_llama33_70B[['final_context_pmids', 'respond_llm', 'pmid_contexts_per_iteration', 'pmid_changes_per_iteration']].loc[0]

final_context_pmids                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [18]:
exploded_res_df_mini_sample_3_pair_k_5_max_itr4_llama33_70B[[ 'respond_llm','pmid_contexts_per_iteration', 'pmid_changes_per_iteration']].loc[1]

respond_llm                    Based on the provided abstracts, an indirect relationship between temozolomide and IRF1 protein, human, can be proposed as follows:\n\n1. **Temozolomide's mechanism of action**: Temozolomide is a DNA alkylating agent that works by damaging the DNA of cancer cells, thereby inhibiting their growth and proliferation. The effectiveness of temozolomide is influenced by the activity of the O6-methylguanine-DNA methyltransferase (MGMT) enzyme, which can repair the DNA damage caused by temozolomide.\n\n2. **MGMT and IRF1 connection**: The abstract "Methionine-stress: a pleiotropic approach in enhancing the efficacy of chemotherapy" mentions that methionine stress can inhibit the growth of brain tumor xenografts and enhance the efficacy of DNA alkylating chemotherapeutic agents, including temozolomide. This is associated with the upregulation of IRF-1 (Interferon Regulatory Factor 1) among other factors. IRF-1 is a transcription factor involved in the regulation o

In [ ]:
# output_file = process_and_save_dataframe_with_llm(
#     df=df,
#     sents_db=sents_db,
#     max_iterations=3,
#     k=3,
#     output_dir="./all_pkl_files"
# )
# print(f"Output saved at: {output_file}")

In [ ]:
res_df_774 = pd.read_pickle('./all_pkl_files/proc_774_samp_6.pkl')
res_df_774

In [5]:
res_df_774 = pd.read_pickle('./all_pkl_files/proc_774_samp_6.pkl')
res_df_774

,pair,subj_name,obj_name,pmid,pmid_text,source,first_year,label,context_pmids,path,context_pmids_space_edge,sorted_context_pmids,final_context_pmids,respond_llm,respond_time_llm,prompts_per_iteration,pmid_contexts_per_iteration,pmid_changes_per_iteration
774,C0076080 C1530270,temozolomide,"IRF1 protein, human",35274103,XAF1 directs glioma response to temozolomide t...,sr_pred_sent_7,2022,1,"{('C0076080', 'C0069197'): ['19514084', '32901...","[C0076080, C0069197, C1530270]","{('C0076080', 'C0069197'): ['4300795', '210665...","{('C0076080', 'C0069197'): ['19514084', '32901...","{('C0076080', 'C0069197'): ['19514084', '32901...","[{'iteration': 1, 'response': 'Based on the pr...","[28.262667179107666, 40.4062659740448, 32.7980...","[Based on the following scientific abstracts, ...","[{('C0076080', 'C0069197'): ['19514084', '3290...","[{'edge': ('C0069197', 'C1530270'), 'dropped':..."


In [8]:
res_df_774[['respond_llm',	'respond_time_llm',	'prompts_per_iteration',	'pmid_contexts_per_iteration',	'pmid_changes_per_iteration']]



respond_llm  \
774  [{'iteration': 1, 'response': 'Based on the provided scientific abstracts, an indirect relationship between temozolomide and IRF1 protein, human, can be proposed as follows:

1. **Temozolomide's mechanism of action**: Temozolomide is a DNA alkylating agent that works by damaging the DNA of cancer cells, thereby inhibiting their growth and proliferation. The effectiveness of temozolomide is influenced by the activity of the O6-methylguanine-DNA methyltransferase (MGMT) enzyme, which can repair the DNA damage caused by temozolomide.

2. **MGMT and IRF1 connection**: The abstract "Methionine-stress: a pleiotropic approach in enhancing the efficacy of chemotherapy" mentions that methionine stress (MET-stress) can inhibit the growth of brain tumor xenografts and enhance the efficacy of DNA alkylating chemotherapeutic agents, including temozolomide. MET-stress is associated with the upregulation of IRF-1, among other genes. IRF-1 is a transcription factor that plays a role in regulating cell growth, differentiation, and apoptosis.

3. **MET-stress and MGMT**: The same abstract states that MET-stress is associated with the marked inhibition of MGMT expression. As MGMT is a key enzyme involved in repairing DNA damage caused by temozolomide, its inhibition can enhance the effectiveness of temozolomide.

4. **Temozolomide and IRF1 indirect relationship**: Given that MET-stress can upregulate IRF-1 and inhibit MGMT, it is possible that temozolomide treatment can indirectly influence IRF-1 expression through the MET-stress pathway. If temozolomide treatment induces MET-stress in cancer cells, this could lead to the upregulation of IRF-1, which in turn could contribute to the anticancer effects of temozolomide.

In summary, the indirect relationship between temozolomide and IRF1 protein, human, can be proposed as follows: temozolomide → MET-stress → MGMT inhibition → IRF-1 upregulation. This relationship suggests that temozolomide treatment can indirectly influence IRF-1 expression through the MET-stress pathway, which may contribute to its anticancer effects. However, further research is needed to fully elucidate the mechanisms underlying this relationship.'}, {'iteration': 2, 'response': 'To describe an indirect relationship between temozolomide and IRF1 protein, human, we must consider the key findings and mechanisms mentioned in the provided abstracts, even though none directly mention IRF1. IRF1 (Interferon Regulatory Factor 1) is a protein involved in the regulation of the immune system, cell growth, and differentiation, and it plays a role in the response to viral infections and in the regulation of apoptosis.

1. **Temozolomide's Mechanism and Resistance**: Temozolomide is an alkylating agent used in the treatment of glioblastoma. It works by methylating DNA, which leads to DNA damage and subsequently triggers cell death. However, its efficacy can be limited by the activity of the DNA repair enzyme O6-methylguanine-DNA methyltransferase (MGMT), which can repair the DNA damage induced by temozolomide. The first abstract discusses the correlation between MGMT promoter methylation status and the survival benefit of temozolomide treatment in elderly patients with glioblastoma.

2. **PARP-1 Inhibition and Temozolomide Sensitivity**: The second abstract explores how PARP-1 (Poly (ADP-ribose) polymerase 1) inhibition can sensitize glioblastoma cells to temozolomide, independent of MGMT activity. PARP-1 is involved in DNA repair, and its inhibition can lead to an accumulation of DNA damage, making cells more susceptible to the effects of temozolomide.

3. **Multimodal Treatment Approaches**: The third abstract discusses a multimodal treatment approach combining local chemotherapy with carmustine wafers and concomitant radiochemotherapy with temozolomide for newly diagnosed glioblastoma patients. This approach aims to utilize the sensitizing effects of temozolomide on MGMT and potentially other pathways to overcome drug 